In [3]:
#Installs tensor flow
pip install tensorflow

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
  Using cached tensorflow-2.8.0-cp37-cp37m-manylinux2010_x86_64.whl (497.5 MB)
  Using cached tf_estimator_nightly-2.8.0.dev2021122109-py2.py3-none-any.whl (462 kB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached flatbuffers-2.0-py2.py3-none-any.whl (26 kB)
  Using cached libclang-13.0.0-py2.py3-none-manylinux1_x86_64.whl (14.5 MB)
  Using cached tensorflow_io_gcs_filesystem-0.24.0-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (2.1 MB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached grpcio-1.44.0-cp37-cp37m-manyli

In [10]:
import os
import json
import sagemaker
import numpy as np
import pandas as pd
import tensorflow as tf
from sagemaker import get_execution_role
from sagemaker.tensorflow import TensorFlow

# Gets number of users and pages from stored data (data prep)
%store -r n_user
%store -r n_item

# Get bucket/sagemaker session data
sage_session = sagemaker.Session()
role = get_execution_role()
bucket = sage_session.default_bucket()

# Location for training dataset
training_uri = os.path.join(f's3://{bucket}', 'data')

# Sagemaker instance data subject to change after more research
num_of_instance = 1
instance_type = 'ml.c5.2xlarge'
train_script = "script.py"

# Some base parameters too be tweaked later
training_parameters = {
    'epochs': 1,
    'batch_size': 256,
    'n_user': n_user,
    'n_item': n_item
}

# training framework specs
tensorflow_version = '2.1.0'
python_version = 'py3'
distributed_training_spec = {'parameter_server': {'enabled': True}}

# Instantiate training job
ncf_estimator = TensorFlow(
    entry_point=train_script,
    role=role,
    train_instance_count=num_of_instance,
    train_instance_type=instance_type,
    framework_version=tensorflow_version,
    py_version=python_version,
    distributions=distributed_training_spec,
    hyperparameters=training_parameters
)

# Start training
ncf_estimator.fit(training_uri)

# Deploying of training model so it can be called
endpoint_name = 'odb-recommendation-engine'

predictor = ncf_estimator.deploy(initial_instance_count=1,
                                 instance_type='ml.c5.xlarge',
                                 endpoint_type='tensorflow-serving',
                                 endpoint_name=endpoint_name)


distributions has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


2022-03-14 18:23:17 Starting - Starting the training job...
2022-03-14 18:23:43 Starting - Preparing the instances for trainingProfilerReport-1647282197: InProgress
......
2022-03-14 18:24:43 Downloading - Downloading input data......
2022-03-14 18:25:45 Training - Training image download completed. Training in progress..2022-03-14 18:25:49,681 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2022-03-14 18:25:49,688 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2022-03-14 18:25:49,968 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2022-03-14 18:25:49,982 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2022-03-14 18:25:49,996 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2022-03-14 18:25:50,004 sagemaker-containers INFO     Invoking user script
Training Env:
{
    "additional_framework_parameters": {
        "sagemaker_parameter

UnexpectedStatusException: Error for Training job tensorflow-training-2022-03-14-18-23-17-356: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
Command "/usr/bin/python3 script.py --batch_size 256 --epochs 1 --model_dir s3://sagemaker-us-east-1-350614259618/tensorflow-training-2022-03-14-18-23-17-356/model --n_item 797 --n_user 1660", exit code: 1